In [1]:
import csv
import math
import statistics
import benchmarks
import numpy as np
import pandas as pd
from scipy.special import _logsumexp

### Functions that will be called in the main code

In [33]:
def fitness(pop: list, n: int, dimension: int, upper_b: int, lower_b: int, objf: object) -> list:
    fit1 = np.zeros((n,1))
    
    for i in range(len(pop)):
        fit1[i] =  objf(pop[i,:])
        #print("Valor de fit1 -> ",fit1[i],"\n")
        #print(i)
    
    #masked_fit = np.nan_to_num(fit1)
    #best fit and position
    best_fit = np.min(fit1)
    pos = np.argmin(fit1)
    
    return fit1, best_fit, pos

In [3]:
def herding(pop: list, Vt: list, fit: list, n: int, dimension: int, acc: list, time: list) -> list:
    
    fit1 = np.sort(fit, axis=0,kind="heapsort") 
    idx = np.argsort(fit, axis=0)
    #print("Fit ordenado -> ",fit1,"\nIndice ordenacao -> ",idx,"\n")
    pop1 = np.zeros((n, dimension))
    Vt1 = np.zeros((n, dimension))
    acc1 = np.zeros((n, dimension))
    time1 = np.zeros((n, dimension))

    for i in range(n):
        
        pop1[i, :] = pop[idx[i], :]
        Vt1[i, :] = Vt[idx[i], :]
        #print("popSlice -> ",pop[idx[i], :],"\nVtSlice -> ", Vt[idx[i], :],"\n")
        acc1[i, :] = acc[i, :]
        time1[i] = time[i]
    
    return pop1, Vt1, fit1, acc1, time1

In [4]:
def generate(n: int, dimension: int, upper_b: int, lower_b: int) -> list:
    boundary_no = 1
    pop = np.zeros((n, dimension))
    
    if boundary_no == 1:
        pop = np.random.random(size= (n, dimension)) * (upper_b - lower_b) + lower_b
        
    #Unused part of the algorithm, that's why it's commented
    #if boundary_no > 1:
    #    for i in range(1,dimension):
    #        pop = np.random.random(size = (n,1))*(upper_b-lower_b)+lower_b;
    x = np.random.random(size = (n, dimension))
    M = pop

    return M, x


In [5]:
def update(Vt: list, time: list, acc: list, n: int, dimension: int, eye: int) -> list:
    pop1 = np.zeros((n,dimension))
    for i in range(n):
        for j in range(dimension):
            #Updating the position of dogs
            if (i <= 3):
                pop1[i][j] = Vt[i][j] * time[i] + (1/2) * acc[i][j] * (np.power(time[i], 2))
            
            #Updating position of sheep
            if (i > 3):
                if eye == 1:
                    pop1[i][j] = Vt[i][j] * time[i] - (1/2) * acc[i][j] * (np.power(time[i], 2))
                else:
                    pop1[i][j] = Vt[i][j] * time[i] + (1/2) * acc[i][j] * (np.power(time[i], 2))
            
    return pop1

In [99]:
def update_v(Vt: list, n: int, dimension: int, acc: list, time: list, pop: list, fit: list, eye: int) -> list:
    #Choosing left and right dogs
    right_dog = np.random.randint(2,3)
    if (right_dog == 2):
        left_dog = 3
    else:
        left_dog = 2

    acc1 = np.ones((n,dimension))
    time1 = np.ones((n,1))
    Vt1 = np.copy(Vt)
    r = len(acc)
    l = len(acc)
    acc2 = np.zeros((r, l))
    fit1 = fit[l-1]

    #Finding Dg value to choose which sheep to gather and which to stalk
    fit2 = (fit[1] + fit[2]) / 2
    f = 0
    tempg = 0
    temps = 0

    #Setting parameters for eyeing
    if eye == 1:
        if fit[right_dog] < fit[left_dog]:
            acc2[left_dog, :] = -1 * acc[left_dog, :]
            f = left_dog
        else:
            acc2[right_dog, :] = -1 * acc[right_dog, :]
            f = right_dog

    for i in range(n):
        for j in range(dimension):
            #Velocity updation of dogs
            if (i <= 3):
                Vt1[i][j] = math.sqrt(np.power(Vt[i][j], 2) + (2 * acc[i][j]) * abs(pop[i][j]))
                
            #Velocity updation of sheep
            if (i > 3):
                if eye == 1:
                    Vt1[i][j] = math.sqrt(np.power(Vt1[f][j], 2) + (2*acc2[f][j]) * abs(pop[i][j]))

                else:
                    #Velocity updation of gathered sheep
                    if (fit1 - fit[i] > fit2 - fit[i]):
                        Vt1[i][j] = math.sqrt(np.power(Vt1[0,j], 2) + (2*acc[0,j]))
                        #tempg[i] = i        
                
                    #Velocity updation of stalked sheep
                    if (fit1 - fit[i] <= fit2 - fit[i]):
                        Vt1[i][j] = math.sqrt(np.power(Vt[right_dog][j]*math.tan(np.random.randint(1,89)), 2) + (2*acc[right_dog][j]*abs(pop[right_dog][j]))) + math.sqrt(np.power(Vt[left_dog][j]*math.tan(np.random.randint(91,179)), 2) + (2*acc[left_dog][j]*abs(pop[left_dog][j])))
                        Vt1[i][j] = (Vt1[i][j]) / 2
                        #temps[i] = i
    
    #Updation of time and acceleration
    for i in range(n):
        s = 0
        for j in range(dimension):
            acc1[i][j] = abs((Vt1[i][j]) - (Vt[i][j])) / (time[i][j])
            s += (Vt1[i][j] - Vt[i][j]) / acc1[i][j]
            
        time1[i] = abs(np.mean(s))

    return Vt1, acc1, time1, right_dog, left_dog, tempg, temps

In [95]:
def check(pop: list, n: int, dimension: int, upper_b: int, lower_b: int, acc: list, Vt: list, time: list) -> list:
    pop1 = np.copy(pop)
    acc1 = np.copy(acc)
    time1 = np.copy(time)
    Vt1 = np.copy(Vt)
    for i in range(n):
        for j in range(dimension):

            if (pop[i][j] >= upper_b or pop[i][j] <= lower_b or pop[i][j] == 0):
                pop1[i][j] = np.random.random() * ((upper_b - lower_b) + lower_b)
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()
    
    for i in range(n):
        for j in range(dimension):
            if (math.isnan(acc[i][j]) == 1 or acc[i][j] == 0):
                pop1[i][j] = np.random.random() * ((upper_b - lower_b) + lower_b)
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()

    for i in range(n):
        for j in range(dimension):
            if (math.isnan(Vt[i][j]) == 1 or Vt[i][j] == 0):
                pop1[i][j] = np.random.random() * ((upper_b - lower_b) + lower_b)
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()

    for i in range(n):
        for j in range(dimension):
            if (math.isnan(time1[i]) == 1 or time1[i] == 0):
                pop1[i][j] = np.random.random() * ((upper_b - lower_b) + lower_b)
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()


    return pop1, acc1, time1, Vt1

### Creation of a CSV file to get the benchmark results

In [108]:
data_bench = "../data/benchmark.csv"
standard = 0
f_worst = 0
f_min = math.inf
bench_mean = 0
bench_std = []
fields = ['Optima', 'MaxFitness', 'Mean']

### Main code

In [109]:
#Population size
n = 30

#No. of rounds to colect the benchmark
round_times = 30

#Maximum no. of iterations
gen = 200

#Optimization function name
fobj, lower_b, upper_b, dim = benchmarks.get_func_details(0)

#Intialize the population(init_p-Population,acc-acceleration of each individual)
init_p, acc = generate(n, dim, upper_b, lower_b) # OK
#Vt = velocity of each individuals
Vt = np.zeros((n, dim))

#Time of each individual
time = np.random.random(size = (n,1))
#Max fitness value
fopt = math.inf
#Variable to store fitness
fit = np.zeros((n, 1))
pop = init_p
#k = counter variable for iterations  required for Eyeing mechanism
k = 1
fopt_1 = np.zeros((gen, 1))
with open(data_bench, 'w') as csvfile:
    csvwriter = csv.DictWriter(csvfile, fieldnames=fields)
    csvwriter.writeheader()
    for i in range(round_times):
        for g in range(gen):
            #Calculate fitness of indivuals
            fit, maxf, pos = fitness(pop, n, dim, upper_b, lower_b, getattr(benchmarks, 'func1_op'))

            eye = 0
            if g == 0:
                fopt = maxf
                
            #Finding the optimum fitness value
            if fopt > maxf:
                fopt = maxf
                
            fopt_1[g] = fopt
            if g > 0:
                if fopt_1[g] > fopt_1[g-1]:
                    k += 1
                    if k > 5:
                        eye = 1
                        k = 0
            
            pop, Vt, fit, acc, time = herding(pop, Vt, fit, n, dim, acc, time)
                
            Vt, acc, time, r1, l1, tempg, temps = update_v(Vt, n, dim, acc, time, pop, fit, eye)
            
            pop = update(Vt, time, acc, n, dim, eye)
            
            pop, acc, time, Vt = check(pop, n, dim, upper_b, lower_b, acc, Vt, time)

            #print("fopt  %.4f\t maxf %.4f\t gen %d"%(fopt, maxf, g))
            if (g == gen-1):
                print(i)
                bench_mean += fopt
                bench_std.append(fopt)
                if (fopt < f_min):
                    f_min = fopt
                if (abs(maxf) > f_worst):
                    f_worst = maxf
            
    bench_mean /= round_times
    #standard = statistics.stdev(bench_std)
    bench_dict = [{'Optima':f_min, 'MaxFitness':f_worst, 'Mean': bench_mean}]        
    csvwriter.writerows(bench_dict)
    csvfile.close()
    

fopt  68673.5702	 maxf 68673.5702	 gen 0
fopt  63562.4066	 maxf 63562.4066	 gen 1
fopt  21205.8656	 maxf 21205.8656	 gen 2
fopt  746.4838	 maxf 746.4838	 gen 3
fopt  746.4838	 maxf 872.4021	 gen 4
fopt  10.2290	 maxf 10.2290	 gen 5
fopt  10.2290	 maxf 8125.5889	 gen 6
fopt  10.2290	 maxf 36256.2678	 gen 7
fopt  10.2290	 maxf 25369.7464	 gen 8
fopt  10.2290	 maxf 2027.6154	 gen 9
fopt  10.2290	 maxf 62512.2898	 gen 10
fopt  10.2290	 maxf 17119.7331	 gen 11
fopt  10.2290	 maxf 58618.0500	 gen 12
fopt  10.2290	 maxf 70584.0857	 gen 13
fopt  10.2290	 maxf 14636.5464	 gen 14
fopt  10.2290	 maxf 17034.2741	 gen 15
fopt  10.2290	 maxf 55330.7061	 gen 16
fopt  10.2290	 maxf 223.4425	 gen 17
fopt  10.2290	 maxf 43.8774	 gen 18
fopt  10.2290	 maxf 39782.6485	 gen 19
fopt  10.2290	 maxf 5484.2784	 gen 20
fopt  10.2290	 maxf 2659.4066	 gen 21
fopt  0.0000	 maxf 0.0000	 gen 22
fopt  0.0000	 maxf 0.0000	 gen 23
fopt  0.0000	 maxf nan	 gen 24
fopt  0.0000	 maxf nan	 gen 25
fopt  0.0000	 maxf 71659.22

C:\Users\artur\AppData\Local\Temp/ipykernel_16076/1856055301.py:60: RuntimeWarning: invalid value encountered in double_scalars
  s += (Vt1[i][j] - Vt[i][j]) / acc1[i][j]


fopt  0.0000	 maxf 1386.2508	 gen 28
fopt  0.0000	 maxf 10106.8823	 gen 29
fopt  0.0000	 maxf 76698.4504	 gen 30
fopt  0.0000	 maxf 141.8523	 gen 31
fopt  0.0000	 maxf 0.0731	 gen 32
fopt  0.0000	 maxf 146.9998	 gen 33
fopt  0.0000	 maxf 64879.1427	 gen 34
fopt  0.0000	 maxf 53851.0445	 gen 35
fopt  0.0000	 maxf 3258.3262	 gen 36
fopt  0.0000	 maxf 70590.0702	 gen 37
fopt  0.0000	 maxf 70231.2267	 gen 38
fopt  0.0000	 maxf 678.7779	 gen 39
fopt  0.0000	 maxf 19998.7310	 gen 40
fopt  0.0000	 maxf 802.0786	 gen 41
fopt  0.0000	 maxf 4331.1128	 gen 42
fopt  0.0000	 maxf 29291.8321	 gen 43
fopt  0.0000	 maxf 23733.7277	 gen 44
fopt  0.0000	 maxf 63212.8087	 gen 45
fopt  0.0000	 maxf 4115.5462	 gen 46
fopt  0.0000	 maxf 78315.9759	 gen 47
fopt  0.0000	 maxf 64111.0158	 gen 48
fopt  0.0000	 maxf 69803.2750	 gen 49
fopt  0.0000	 maxf 7801.6030	 gen 50
fopt  0.0000	 maxf 3949.4106	 gen 51
fopt  0.0000	 maxf 0.0000	 gen 52
fopt  0.0000	 maxf 0.0000	 gen 53
fopt  0.0000	 maxf nan	 gen 54
fopt  0